In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.externals import joblib
from collections import Counter 

In [3]:
def get_feature(line,label):
    num_len=0
    capital_len=0
    key_num=0
    feature3=0
    num_len=len(re.compile(r'\d').findall(line))
    if len(line)!=0:
        num_f=num_len/len(line)#数字字符频率
    else:
        num_f=0
    capital_len=len(re.compile(r'[A-Z]').findall(line))
    if len(line)!=0:
        capital_f=capital_len/len(line)#大写字母频率
    else:
        capital_f=0
    line=line.lower()

    key_num=line.count('and')+line.count('or')+line.count('xor')+line.count('sysobjects')+line.count('version')+line.count('substr')+line.count('len')+line.count('substring')+line.count('exists')
    key_num=key_num+line.count('union')+line.count('asc')+line.count('inner join')+line.count('xp_cmdshell')+line.count('version')+line.count('exec')+line.count('having')+line.count('unnion')+line.count('order')+line.count('information schema')
    key_num=key_num+line.count('load_file')+line.count('load data infile')+line.count('into outfile')+line.count('into dumpfile')
    if len(line)!=0:
        space_f=(line.count(" ")+line.count("%20"))/len(line)#空格百分比
        special_f=(line.count("{")*2+line.count('28%')*2+line.count('NULL')+line.count('[')+line.count('=')+line.count('?'))/len(line)
        prefix_f=(line.count('\\x')+line.count('&')+line.count('\\u')+line.count('%'))/len(line)
    else:
        space_f=0
        special_f=0
        prefix_f=0
    #print('%f,%f,%f,%f,%f,%f,%f,%f' % (len(line),key_num,capital_f,num_f,space_f,special_f,prefix_f,label))

    return len(line),key_num,capital_f,num_f,space_f,special_f,prefix_f,label

In [4]:
def generate(odir,wdir,label):
    f_input=open(wdir, 'w')
    with open(odir, 'rb') as f:
        data = [x.decode('utf-8').strip() for x in f.readlines()]
        #print(data)
        for line in data:
            f_input.write('%f,%f,%f,%f,%f,%f,%f,%f' % (get_feature(line,label))+'\n')

    f_input.close()
    return wdir

In [5]:
sql_matrix=generate("./data/sqlnew.csv","./data/sql_matrix.csv",1)
nor_matrix=generate("./data/normal_less.csv","./data/nor_matrix.csv",0)

In [6]:
df = pd.read_csv(sql_matrix)
df.to_csv("./data/all_matrix.csv",encoding="utf_8_sig",index=False)
df = pd.read_csv( nor_matrix)
df.to_csv("./data/all_matrix.csv",encoding="utf_8_sig",index=False, header=False, mode='a+')

In [7]:
from sklearn import tree
import sklearn.ensemble as ek
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

D:\python\Anaconda\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
feature_max = pd.read_csv('./data/all_matrix.csv')
arr=feature_max.values
np.random.shuffle(arr)#打乱顺序
data = np.delete(arr, -1, axis=1) #删除最后一列
#print(arr)
target=arr[:,7]

In [9]:
data[0]

array([10. ,  1. ,  0. ,  0.2,  0.2,  0.1,  0. ])

In [10]:
target

array([1., 0., 1., ..., 0., 0., 1.])

In [11]:
#随机划分训练集和测试集
train_data,test_data,train_target,test_target = train_test_split(data,target,test_size=0.3,random_state=3)


In [17]:
#训练集中正负样本的数量
Counter(train_target)

Counter({1.0: 3452, 0.0: 3529})

In [18]:
#测试集中正负样本的数量
Counter(test_target)

Counter({1.0: 1523, 0.0: 1470})

In [12]:
len(train_target)+len(test_target)

9974

In [13]:
model = { "DecisionTree    ":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest     ":ek.RandomForestClassifier(n_estimators=10,max_depth=2),
         "Adaboost         ":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting ":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB              ":GaussianNB(),
         "LogisticRegression":LogisticRegression()   
}

In [20]:
#测试不同算法的准确度
print("算法        \t score_test    \t score_train    \t 精确率  \t  召回率 \t 混淆矩阵")
#print("算法        \t score_test    \t score_train    \t 精确率  \t  召回率 ")
for algo in model:
    clf = model[algo]
    clf.fit(train_data,train_target)
    score_test = clf.score(test_data,test_target)
    score_train = clf.score(train_data,train_target)
   
    y_pred=clf.predict(test_data)#预测
    precision=metrics.precision_score(y_true=test_target,y_pred=y_pred)
    recall=metrics.recall_score(y_true=test_target,y_pred=y_pred)
    confusion=metrics.confusion_matrix(y_true=test_target,y_pred=y_pred)
    joblib.dump(clf, './data/'+algo+'.model')
    print(("%s \t %.6s \t %.6s \t %.6s \t %.6s \t %s  ")%(algo,score_test,score_train,precision,recall,confusion))
    #print(("%s \t %.6s \t %.6s \t %.6s \t %.6s \t  ")%(algo,score_test,score_train,precision,recall))

算法        	 score_test    	 score_train    	 精确率  	  召回率 	 混淆矩阵
DecisionTree     	 0.9939 	 0.9997 	 0.9980 	 0.9901 	 [[1467    3]
 [  15 1508]]  
RandomForest      	 0.9799 	 0.9769 	 0.9972 	 0.9632 	 [[1466    4]
 [  56 1467]]  
Adaboost          	 0.9956 	 0.9979 	 0.9986 	 0.9927 	 [[1468    2]
 [  11 1512]]  
GradientBoosting  	 0.9939 	 0.9975 	 0.9966 	 0.9914 	 [[1465    5]
 [  13 1510]]  
GNB               	 0.9766 	 0.9785 	 0.9795 	 0.9743 	 [[1439   31]
 [  39 1484]]  
LogisticRegression 	 0.9201 	 0.9067 	 0.9234 	 0.9192 	 [[1354  116]
 [ 123 1400]]  


In [15]:
print("算法           \t  预测结果")
for algo in model:
    clf =joblib.load('./data/'+algo+'.model') 
    feature = get_feature("select *",0)
    feature = [feature[:7]]
    y_pred=clf.predict(feature)#预测
    print(("%s \t %.6s ")%(algo,y_pred))


算法           	  预测结果
DecisionTree     	 [1.] 
RandomForest      	 [1.] 
Adaboost          	 [1.] 
GradientBoosting  	 [1.] 
GNB               	 [1.] 
LogisticRegression 	 [1.] 
